In [ ]:
!pip install enzope --upgrade

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import os
import enzope
import networkx as nx
from tqdm import tqdm

In [ ]:
def r1(x):
    return x**(-1)

def print_info(gtgs):
    cons = [gtgs[i].get_mean_degree()[0] for i in range(len(gtgs))]
    clust = [[nx.average_clustering(gtgs[i].G) for i in range(len(gtgs))]]
    assort = [nx.degree_assortativity_coefficient(gtgs[i].G) for i in range(len(gtgs))]
    print(f"Mean k: {np.mean(cons):.2f} +- {np.std(cons):.2f}")
    print(f"Mean clustering: {np.mean(clust):.2f} +- {np.std(clust):.2f}")
    print(f"Mean assortativity: {np.mean(assort):.2f} +- {np.std(assort):.2f}")

In [ ]:
# Params
reps = 100
n_nodes = 1000
theta = 65
mcs = 200000

In [ ]:
def graph_ensemble():
    gtgs = [enzope.graphs.graph_class.GTG(n_nodes=n_nodes, theta=theta, join='add', p_dist=r1) for i in range(reps)]
    print(f'theta={theta}')
    print_info(gtgs)
    return gtgs

theta=65
Mean k: 4.27 +- 0.33
Mean clustering: 0.54 +- 0.03
Mean assortativity: -0.13 +- 0.03


In [ ]:
for f in [0, .1]:
    graphs = graph_ensemble()
    ensemble = enzope.GPUEnsemble(n_models=reps, n_agents=n_nodes, graphs=gtgs, f=0)

    for model, rng_state in tqdm(zip(ensemble.models, ensemble.rng_states)):
        model.MCS(mcs, ensemble.tpb, ensemble.bpg, rng_state)


    ginis = [enzope.measures.gini(model.w) for model in ensemble.models]
    print(f"{f}, {np.mean(ginis)}, {np.std(ginis)},")


100it [01:46,  1.07s/it]


In [ ]:
ensemble.models[0]
ginis = [enzope.measures.gini(model.w) for model in ensemble.models]

### Corro los otros grafos

In [ ]:
!pip install enzope --upgrade

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import os
import enzope
import networkx as nx
from tqdm import tqdm
from numba import cuda
import time

def r1(x):
    return x**(-1)

def print_info(gtgs):
    cons = [gtgs[i].get_mean_degree()[0] for i in range(len(gtgs))]
    clust = [[nx.average_clustering(gtgs[i].G) for i in range(len(gtgs))]]
    assort = [nx.degree_assortativity_coefficient(gtgs[i].G) for i in range(len(gtgs))]
    print(f"Mean k: {np.mean(cons):.2f} +- {np.std(cons):.2f}")
    print(f"Mean clustering: {np.mean(clust):.2f} +- {np.std(clust):.2f}")
    print(f"Mean assortativity: {np.mean(assort):.2f} +- {np.std(assort):.2f}")

# Params
reps = 10
n_nodes = 1000
theta = 65
mcs = 200000

def ba_ensemble():
    # BA graphs
    # <k>: 3.992 +- 0.0000000000
    # C: 0.026 +- 0.005
    # r: -0.092 +- 0.011
    return [nx.barabasi_albert_graph(n_nodes, 2) for _ in range(reps)]

def er_ensemble():
    # ER graphs
    # <k>: 3.992 +- 0.0000000000
    # C: 0.003 +- 0.000
    # r: -0.001 +- 0.000
    return [nx.erdos_renyi_graph(n_nodes, 0.00405) for _ in range(reps)]

def graph_ensemble():
    gtgs = [enzope.graphs.graph_class.GTG(n_nodes=n_nodes, theta=theta, join='add', p_dist=r1) for i in range(reps)]
    return gtgs


# Esto esta copiado con distintos f_set porque el cluster me reta que uso muchos recursos
f_set = np.arange(0.26, .501, 0.02)
ensemble = [None for _ in range(len(f_set))]

vals = {'f':[], 'gini':[], 'std':[]}

for j, f in enumerate(f_set):
    # graphs = er_ensemble()

    # Esto es un fix para que corra el programa, ya que la funcion get_neighbours_gpu solo esta en la clase GTG
    gtgs = [enzope.graphs.graph_class.GTG(n_nodes, theta=10000) for _ in range(reps)]
    # for i in range(reps):
        # gtgs[i].G = graphs[i]

    ensemble[j] = enzope.GPUEnsemble(n_models=reps, n_agents=n_nodes, graphs=None, f=f)

    for model, rng_state in tqdm(zip(ensemble[j].models, ensemble[j].rng_states)):
        model.MCS(mcs, ensemble[j].tpb, ensemble[j].bpg, rng_state)

    ginis = [enzope.measures.gini(model.w) for model in ensemble[j].models]

    print(f"{f}, {np.mean(ginis)}, {np.std(ginis)},")
    vals['f'].append(f)
    vals['gini'].append(np.mean(ginis).copy())
    vals['std'].append(np.std(ginis).copy())


    # cuda.current_context().memory_manager.deallocations.clear()



10it [00:27,  2.72s/it]


0.26, 0.7116012512207031, 0.01360986593125381,


10it [00:27,  2.75s/it]


0.28, 0.6862101593017577, 0.014042720253369596,


10it [00:29,  2.91s/it]


...

0.4600000000000002, 0.5069606475830077, 0.014348736693578319,


10it [00:33,  3.36s/it]


0.4800000000000002, 0.4803326934814452, 0.014448819079831424,


10it [00:33,  3.36s/it]

0.5000000000000002, 0.4707265380859374, 0.014241306020250167,


BA:
f,gini,std_gini
0.0, 0.7816632705688475, 0.01777065905305278
0.02, 0.815324296569824, 0.017095978898143332
0.04, 0.8106165710449218, 0.0158738874179976
0.06, 0.8165143249511717, 0.015090044139978609
0.08, 0.814198387145996, 0.013421036305127908
0.1, 0.8068092132568359, 0.011088948830621902
0.12, 0.7831787948608397, 0.01833854286890259
0.14, 0.7698057144165038, 0.010803953053703191
0.16, 0.7587137634277343, 0.010332630019980086
0.18, 0.7379317138671875, 0.019990296680831125
0.19999999999999998, 0.7090724227905272, 0.02544552862919703
0.21999999999999997, 0.6980368194580077, 0.014679612113204643
0.23999999999999996, 0.6763318939208982, 0.02596522613172248
0.25999999999999995, 0.6553551055908201, 0.01783156614875531
0.2799999999999999, 0.6301252929687499, 0.015359553291672375
0.29999999999999993, 0.6052985443115234, 0.026795911711782094
0.31999999999999995, 0.5745525115966796, 0.029189065294532267
0.3399999999999999, 0.5492464477539061, 0.030741271594571885
0.36, 0.5392178344726561, 0.019606390954000377
0.38, 0.5137534576416015, 0.01797667451362144
0.4, 0.5131772064208983, 0.016126501605276136
0.42000000000000004, 0.4806566650390624, 0.024499123459637395
0.44000000000000006, 0.47044121093749985, 0.02289733012798621
0.4600000000000001, 0.4560621337890624, 0.022172119474659337
0.4800000000000001, 0.45055175781249995, 0.02297353797211971
0.5, 0.43242620239257806, 0.021976786200419405



er:
f,gini,std_gini
0.0, 0.8140626327514647, 0.007414035005004858
0.02, 0.8408949478149413, 0.011050080159947216
0.04, 0.8440369674682616, 0.011208377245711517
0.06, 0.8357656051635741, 0.010804063742539982
0.08, 0.8283870803833006, 0.0052585769209538315
0.1, 0.8159758331298826, 0.008280989383099796
0.12, 0.7936572479248045, 0.011569159579138745
0.14, 0.7737708007812498, 0.014593105174054748
0.16, 0.7523505355834962, 0.01379967094501448
0.18, 0.7308268249511718, 0.013838598158584021
0.2, 0.7136759307861327, 0.011202995012426113
0.22, 0.6687569885253906, 0.013693710074900837
0.24, 0.6555496215820311, 0.014575238479818703
0.26, 0.6295745727539062, 0.0104214054357027
0.28, 0.6211477325439452, 0.012973680799841675
0.3, 0.591706787109375, 0.013232386568451682
0.32, 0.5668656677246092, 0.012365797931338879
0.34, 0.5432980224609374, 0.014782429352818587
0.36, 0.5227355010986326, 0.007270983542735113
0.38, 0.5083753875732421, 0.011693260382478489
0.4, 0.48780166320800766, 0.01905045205294629
0.42, 0.4707222351074217, 0.01831407714951808
0.44, 0.4507370788574218, 0.012394115014637254
0.46, 0.4316322021484374, 0.014561140059803869
0.48, 0.43316202392578107, 0.009554177635543986
0.5, 0.4153266052246093, 0.009947173312734621


mean field
f,gini,std_gini
0.0, 0.9982395011663435, 0.0002253046223778311
0.02, 0.993593957042694, 0.001677276973070657
0.04, 0.9552934917449951, 0.004295185623977493
0.06, 0.9320977569580077, 0.005785567210336383
0.08, 0.9163870399475096, 0.00399242001934394
0.1, 0.8897505714416504, 0.008921050739202144
0.12, 0.8663528488159178, 0.006043790147281094
0.14, 0.8399558898925781, 0.00888649786555123
0.16, 0.8262649551391601, 0.009301595102485186
0.18, 0.7997268630981444, 0.01226256488493322
0.2, 0.7810273834228514, 0.014504397531882469
0.22, 0.7441808319091796, 0.011875547358538973
0.24, 0.7280651626586913, 0.014829663815962218
0.26, 0.7116012512207031, 0.01360986593125381
0.28, 0.6862101593017577, 0.014042720253369596
0.30000000000000004, 0.6662096923828124, 0.018067881033706264
0.32000000000000006, 0.6315617797851562, 0.01720388975753711
0.3400000000000001, 0.6175571533203124, 0.017617524158073467
0.3600000000000001, 0.600975259399414, 0.020840615939641788
0.3800000000000001, 0.5765378448486327, 0.017001312338392572
0.40000000000000013, 0.5667791564941406, 0.015687222659820897
0.42000000000000015, 0.5375900726318358, 0.0123554906983234
0.44000000000000017, 0.5169758148193357, 0.012152906401497205
0.4600000000000002, 0.5069606475830077, 0.014348736693578319
0.4800000000000002, 0.4803326934814452, 0.014448819079831424
0.5000000000000002, 0.4707265380859374, 0.014241306020250167
